#### Prerequisites


In [ ]:
from typing import List
from collections import defaultdict
from bisect import bisect_left

## 3158. Find the XOR of Numbers Which Appear Twice

    Difficulty - Easy
    Topic - Array, Bit Manipulation

You are given an array `nums`, where each number in the array appears **either** once or twice.

Return the bitwise `XOR` of all the numbers that appear twice in the array, or 0 if no number appears twice.

**Constraints:**

-   `1 <= nums.length <= 50`
-   `1 <= nums[i] <= 50`
-   Each number in `nums` appears either once or twice.


In [ ]:
class Solution:
    def duplicateNumbersXOR(self, nums: List[int]) -> int:
        ans = 0

        counts = [0] * 51

        for num in nums:
            counts[num] += 1

        for num, count in enumerate(counts):
            if count == 2:
                ans ^= num

        return ans


if __name__ == "__main__":
    sol = Solution()
    cases = [{"nums": [1, 2, 1, 3]}, {"nums": [1, 2, 3]}, {"nums": [1, 2, 2, 1]}]
    for case in cases:
        print(sol.duplicateNumbersXOR(case["nums"]))

## 3159. Find Occurrences of an Element in an Array

    Difficulty - Medium
    Topic - Array

You are given an integer array `nums`, an integer array `queries`, and an integer `x`.

For each `queries[i]`, you need to find the index of the `queries[i]th` occurrence of `x` in the `nums` array. If there are fewer than `queries[i]` occurrences of `x`, the answer should be -1 for that query.

Return an integer array `answer` containing the answers to all queries.

**Constraints:**

-   `1 <= nums.length, queries.length <= 10^5`
-   `1 <= queries[i] <= 10^5`
-   `1 <= nums[i], x <= 10^4`


In [ ]:
class Solution:
    def occurrencesOfElement(
        self, nums: List[int], queries: List[int], x: int
    ) -> List[int]:
        occurrences = [-1] * len(queries)
        indices = [i for i, num in enumerate(nums) if num == x]

        for i, query in enumerate(queries):
            if query <= len(indices):
                occurrences[i] = indices[query - 1]
        return occurrences


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {"nums": [1, 3, 1, 7], "queries": [1, 3, 2, 4], "x": 1},
        {"nums": [1, 2, 3], "queries": [10], "x": 5},
    ]
    for case in cases:
        print(sol.occurrencesOfElement(case["nums"], case["queries"], case["x"]))

## 3160. Find the Number of Distinct Colors Among the Balls

    Difficulty - Medium
    Topics - Array, Hash Maps

You are given an integer `limit` and a 2D array `queries` of size `n x 2`.

There are `limit + 1` balls with **distinct** labels in the range `[0, limit]`. Initially, all balls are uncolored. For every query in `queries` that is of the form `[x, y]`, you mark ball `x` with the color `y`. After each query, you need to find the number of **distinct** colors among the balls.

Return an array `result` of length `n`, where `result[i]` denotes the number of distinct colors after `ith` query.

**Note** that when answering a query, lack of a color _will not_ be considered as a color.

**Constraints:**

-   `1 <= limit <= 10^9`
-   `1 <= n == queries.length <= 10^5`
-   `queries[i].length == 2`
-   `0 <= queries[i][0] <= limit`
-   `1 <= queries[i][1] <= 10^9`


In [ ]:
class Solution:
    def queryResults(self, limit: int, queries: List[List[int]]) -> List[int]:
        ball_colors = {}
        distinct_colors = defaultdict(int)
        result = []

        for ball, color in queries:
            old_color = ball_colors.get(ball)
            if old_color:
                if distinct_colors[old_color] == 1:
                    del distinct_colors[old_color]
                else:
                    distinct_colors[old_color] -= 1

            ball_colors[ball] = color
            distinct_colors[color] += 1
            result.append(len(distinct_colors))

        return result


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {"limit": 4, "queries": [[1, 4], [2, 5], [1, 3], [3, 4]]},
        {"limit": 4, "queries": [[0, 1], [1, 2], [2, 2], [3, 4], [4, 5]]},
    ]
    for case in cases:
        print(sol.queryResults(case["limit"], case["queries"]))

## 3161. Block Placement Queries

    Difficulty - Hard
    Topics - Array, Segment Trees

There exists an infinite number line, with its origin at 0 and extending towards the **positive** x-axis.

You are given a 2D array `queries`, which contains two types of queries:

For a query of type 1, `queries[i] = [1, x]`. Build an obstacle at distance `x` from the origin. It is guaranteed that there is **no** obstacle at distance `x` when the query is asked.
For a query of type 2, `queries[i] = [2, x, sz]`. Check if it is possible to place a block of size `sz` _anywhere_ in the range `[0, x]` on the line, such that the block **entirely** lies in the range `[0, x]`. A block `cannot` be placed if it intersects with any obstacle, but it may touch it. Note that you do `not` actually place the block. Queries are separate.

Return a boolean array `results`, where `results[i]` is `true` if you can place the block specified in the `ith` query of type 2, and `false` otherwise.


In [ ]:
class Solution:
    def getResults(self, queries: List[List[int]]) -> List[bool]:
        bit, res = [0] * (min(50000, 3 * len(queries)) + 1), []

        def bit_range(p, down=True):
            while 0 <= p < len(bit):
                yield p
                p = (p & (p + 1)) - 1 if down else p | (p + 1)

        def get_max(p: int) -> int:
            return max(bit[i] for i in bit_range(p))

        def update(p, val):
            for idx in bit_range(p, False):
                bit[idx] = max(bit[idx], val)

        blocks = [0] + sorted([x for t, x, *sz in queries if t == 1])
        for i in range(1, len(blocks)):
            update(blocks[i], blocks[i] - blocks[i - 1])
        for query in reversed(queries):
            t, x, *sz = query
            p = bisect_left(blocks, x)
            if t == 1:
                if p + 1 < len(blocks):
                    update(blocks[p + 1], blocks[p + 1] - blocks[p - 1])
                del blocks[p]
            else:
                res.append(x - blocks[p - 1] >= sz[0] or get_max(x) >= sz[0])
        return list(reversed(res))


if __name__ == "__main__":
    sol = Solution()
    cases = [
        {"queries": [[1, 2], [2, 3, 3], [2, 3, 1], [2, 2, 2]]},
        {"queries": [[1, 7], [2, 7, 6], [1, 2], [2, 7, 5], [2, 7, 6]]},
        {"queries": [[2, 1, 1]]},
    ]
    for case in cases:
        print(sol.getResults(case["queries"]))